In [1]:
from IPython import get_ipython
kernel = get_ipython()
kernel.config

{'IPKernelApp': {'connection_file': '/Users/Andie.Phan/Library/Jupyter/runtime/kernel-d6e6d7d8-6e95-49b6-a0b2-e765ec03f286.json'}}

In [2]:

kernel_config_app = kernel.config['IPKernelApp']
conn_file = kernel_config_app['connection_file']
kernel_id = conn_file.split('-', 1)[1].split('.')[0]
kernel_id

'd6e6d7d8-6e95-49b6-a0b2-e765ec03f286'

In [3]:
import json
cfg_json = json.load(open(conn_file))
cfg_json

{'shell_port': 57002,
 'iopub_port': 57003,
 'stdin_port': 57004,
 'control_port': 57006,
 'hb_port': 57005,
 'ip': '127.0.0.1',
 'key': '6d96705a-8fe9e9ed81b57af2d4ed33cd',
 'transport': 'tcp',
 'signature_scheme': 'hmac-sha256',
 'kernel_name': ''}

In [4]:
from jupyter_server import serverapp
# from notebook import notebookapp
server_list = []
# server_list += list(notebookapp.list_running_servers())
server_list += list(serverapp.list_running_servers())
server_list

[{'base_url': '/',
  'hostname': 'localhost',
  'password': False,
  'pid': 15441,
  'port': 8888,
  'root_dir': '/Users/Andie.Phan/Projects/otosense/plunk/plunk/ap/notebooks',
  'secure': False,
  'sock': '',
  'token': '32df656aef14e13423ec97b6ba3fe79da260ea9a3996ef99',
  'url': 'http://localhost:8888/',
  'version': '2.6.0'}]

In [5]:
server_cfg = server_list[0]

ws_url = (
    f"ws{server_cfg['url'][4:]}api/kernels/{kernel_id}"
    f"/channels?token={server_cfg['token']}"
)
ws_url

'ws://localhost:8888/api/kernels/d6e6d7d8-6e95-49b6-a0b2-e765ec03f286/channels?token=32df656aef14e13423ec97b6ba3fe79da260ea9a3996ef99'

In [10]:
import asyncio
import websockets
import json

async def execute_code(code, uri=ws_url):
    async with websockets.connect(uri) as websocket:
        # Message to start a new execution request
        msg_start = {
            "header": {
                "msg_id": "execute_request",
                "username": "username",
                "session": "000000000000",
                "msg_type": "execute_request",
            },
            "content": {
                "code": code,
                "silent": False,
                "store_history": True,
                "user_expressions": {},
                "allow_stdin": True,
            },
            "metadata": {},
            "parent_header": {},
        }
        # Sending the message to start code execution
        await websocket.send(json.dumps(msg_start))
        
        while True:
            response = await websocket.recv()
            msg = json.loads(response)
            # Check if the message is an execution result
            if msg["msg_type"] == "execute_result":
                print("Execution Result:", msg["content"]["data"])
                break

# Example usage
code_to_execute = "print('Hello, Jupyter!')"

# Create a new event loop
loop = asyncio.new_event_loop()
asyncio.set_event_loop(loop)

# Run the coroutine in the new event loop
loop.run_until_complete(execute_code(code_to_execute))

# Close the event loop
loop.close()

RuntimeError: Cannot run the event loop while another loop is running

In [ ]:
import sys
sys.executable

'/Users/Andie.Phan/.pyenv/versions/otoframe-3.10/bin/python'

In [8]:
from IPython.display import display, Javascript

# Get the session_id using JavaScript and display it
display(Javascript('IPython.notebook.session.session_id'))

<IPython.core.display.Javascript object>

In [14]:
import websocket
import json
import time

def execute_code(code, uri=ws_url):
    ws = websocket.WebSocketApp(uri)
    
    def on_open(ws):
        # Message to start a new execution request
        msg_start = {
            "header": {
                "msg_id": "execute_request",
                "username": "username",
                "session": "000000000000",
                "msg_type": "execute_request",
            },
            "content": {
                "code": code,
                "silent": False,
                "store_history": True,
                "user_expressions": {},
                "allow_stdin": True,
            },
            "metadata": {},
            "parent_header": {},
        }
        # Sending the message to start code execution
        ws.send(json.dumps(msg_start))
    
    def on_message(ws, message):
        msg = json.loads(message)
        # Check if the message is an execution result
        if msg["msg_type"] == "execute_result":
            print("Execution Result:", msg["content"]["data"])
            ws.close()
    
    ws.on_open = on_open
    ws.on_message = on_message
    
    # Start the WebSocket connection
    ws.run_forever()

# Example usage
code_to_execute = "x = 1 + 2"
execute_code(code_to_execute)

/Users/Andie.Phan/.pyenv/versions/3.10.7/lib/python3.10/ast.py:50: RuntimeWarning: coroutine 'execute_code' was never awaited
  return compile(source, filename, mode, flags,


KeyboardInterrupt: 

In [13]:
x


NameError: name 'x' is not defined